In [2]:
#%pip install -q -r requirements.txt

In [3]:
from dotenv import load_dotenv
import os
from groq import Groq

load_dotenv()

False

In [4]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be attributed to several factors:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in scenarios where timely responses are critical, such as customer support or emergency services.
2. **Low-Latency Requirements**: Many modern applications, like voice assistants, require language models to process and respond to user input within a few hundred milliseconds. Fast language models can meet these low-latency requirements, ensuring a seamless user experience.
3. **Scalability**: Fast language models can handle large volumes of data and user requests, making them essential for large-scale NLP applications, such as language translation platforms or social media analytics tools.
4. **Energy Efficiency**: Fast lang

In [5]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [6]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()


def calculate(operation: str) -> float:
    return eval(operation)


def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [7]:
neil_tyson = Agent(client=client, system=system_prompt)

In [10]:
import re


def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "get_planet_mass"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


In [11]:

#loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?")
#loop(query="You go and shop 12 bananas and 5 apple. The price of bana per unit is 5 penny. Total bill is 1.1 pounds. What is apple per unit?")
#loop(query="You go and shop and spend 20 pounds buying bananas and apples. The 8 bananas cost 0.5 per unit. The the 5 apples cost 0.25 per unit . Now shop the two fruits such that all the menoey is spent. Give how much bananas and apples are bought?")
loop


Thought: Let's start by calculating the total cost of the fruits I've already bought.

Action: calculate: 8 * 0.5 + 5 * 0.25
PAUSE
Observation: 5.25
Thought: I've already spent 5.25 pounds, and I want to spend the remaining 14.75 pounds (20 - 5.25) on more bananas and apples.

Action: calculate: 14.75 / (0.5 + 0.25)
PAUSE
Observation: 19.666666666666668
Thought: This result represents the total number of fruits I can buy with the remaining money. Since I'm buying bananas and apples, I'll divide this number by 2 to find out how many of each fruit I can buy.

Action: calculate: 19.666666666666668 / 2
PAUSE
Observation: 9.833333333333334
Thought: Since I cannot buy a fraction of a fruit, I'll round down to the nearest whole number to find out how many bananas and apples I can buy.

Action: calculate: 9
PAUSE
Observation: 9
Thought: I've reached my answer. I'll buy 9 more bananas and 9 more apples.

Answer: I'll buy 17 bananas (8 + 9) and 14 apples (5 + 9).
